In [ ]:
from pathlib import Path
import yaml
import os

root_dir = root_dir = Path(os.getenv("ROOT_DIR"))
out_base_dir = root_dir / "2024_08_16_A2744_v4" / "glass_niriss"
bin_data_dir = out_base_dir / "binned_data"

repr_seg_path = out_base_dir / "PSF_matched_data" / "repr_seg_map.fits"

with open(out_base_dir / "conv_ancillary_data.yaml", "r") as file:
    info_dict = yaml.safe_load(file)

In [ ]:
from glass_niriss.isophotal import ClusterModels

filt_key = "jwst-nircam-f115w"

print (info_dict[filt_key])

import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

with fits.open(str(info_dict[filt_key]["sci"]).replace("conv","repr")) as hdul:
    plt.imshow(hdul[0].data, vmin=-0.01, vmax=10, origin="lower")

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

sci_path = str(info_dict[filt_key]["sci"]).replace("conv","repr")
sci_path = root_dir / "2024_08_16_A2744_v4" / "tests" / "repr_abell2744clu-grizli-v7.2-f200w-clear_drc_sci.fits"

input_bcgs = Table.read(
    root_dir
    / "2023_11_27__data_products_for_reduction"
    / "ancillary"
    / "matched_cat.fits"
)
bcg_info = Table.read(
    root_dir
    / "2023_11_07_spectral_orders"
    / "Prep"
    / "nis-wfss-ir.cat.fits"
)
# print (bcg)
bcg_coords = SkyCoord.guess_from_table(bcg_info)
match, _, _ = SkyCoord.guess_from_table(input_bcgs).match_to_catalog_sky(bcg_coords)
bcg_info = bcg_info[match]
input_bcgs["FLUX_AUTO"] = bcg_info["FLUX_AUTO"]
input_bcgs.sort("FLUX_AUTO", reverse=True)

out_dir = out_base_dir / "bCG_model_tests"
CM = ClusterModels(
    output_dir=out_dir,
    # ra_key="t"
)

In [ ]:
CM.load_bcg_catalogue(input_bcgs)
CM.load_image(sci_path)

if not (out_dir / "A2744_cluster_models_seg.fits").is_file():
    CM.gen_segmentation()
CM.load_segmentation(out_dir / "A2744_cluster_models_seg.fits")

CM.match_bcgs(plot=True)

In [ ]:
sci_psf = fits.getdata(info_dict[filt_key]["psf"])

from scipy.ndimage import zoom
from astropy.nddata import block_reduce
import astropy.visualization as astrovis
import numpy as np

scale = 10
zoom_in = zoom(sci_psf, scale)
# zoom_out = zoom(zoom_in, 0.25)
zoom_out = block_reduce(zoom_in, scale*2)
zoom_out = block_reduce(np.pad(zoom_in,(5,5)), scale*2)
print (sci_psf.shape, zoom_in.shape, zoom_out.shape,np.pad(zoom_in,(10,10)).shape)

fig, axs = plt.subplots(1,3)
for i, plot_psf in enumerate([sci_psf, zoom_in, zoom_out]):
    # centre_crop = (
    #     slice(plot_psf.shape[0]//2-10, plot_psf.shape[0]//2+10),
    #     slice(plot_psf.shape[1]//2-10, plot_psf.shape[1]//2+10)
    # )
    centre_crop = (slice(0,None),slice(0,None))
    print (np.nansum(plot_psf))
    axs[i].imshow(plot_psf[centre_crop], norm=astrovis.ImageNormalize(plot_psf[centre_crop], interval=astrovis.PercentileInterval(99.9), stretch=astrovis.LogStretch()))

zoom_out /= np.nansum(zoom_out)

reduced_psf = zoom_out

In [ ]:

CM.model_galaxies(psf=reduced_psf)

In [ ]:
CM.model_galaxies()